# Import Modules & Setup

In [ ]:
!pip install openai pandas tqdm python-dotenv

In [3]:
import openai
import csv
import pandas as pd
import time
import random
from tqdm.notebook import tqdm
import sys
import os
from dotenv import load_dotenv

In [ ]:
output_csv_file = '..\\dataset\\sampled_hn_gh_ai_story_gpt_sentiment.csv'

### Initialize OpenAI API

In [ ]:
# Set your OPENAI_API_KEY as an environment variable and then load
load_dotenv()

# Initialize the OpenAI client with your API key
api_key_openai = os.getenv("OPENAI_API_KEY")
client_openai = openai.OpenAI(api_key=api_key_openai)

### Prompt template

In [ ]:
prompt_template = '''
Please perform Sentiment Classification task. Given the story headline from Hacker News, assign a sentiment label expressed by the author towards "Artificial Intelligence (AI)" from ['negative': -1, 'neutral': 0, 'positive': 1]. Return label only and the reasoning in the following format.

Headline: "AI-Vulner": Repo of multiple vulnerable codes generated by LLM
Label:-1
Reason:The headline suggests that there are mutliple vulnerable codes gnerated by LLM, which is an undesirable traits in software engineering.
Headline: "JAVIN" an AI for solving math problem
Label:0
Reason:The headline simply announces the "JAVIN" project as a math problem solver without clear positive implications.
Headline: "TEXIE," an AI self-driving car that makes your life easier.
Label:1
Reason:The headline advertises an AI self-driving project by claiming that it makes your life easier.

Headline: {headline_input}
Label:
Reason:
'''

# Sentiment from GPT

In [ ]:
def get_sentiment_and_reason_openai(title):
    prompt = prompt_template.format(headline_input=title)

    for _ in range(3):
        try:
            response = client_openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are an expert at analyzing text."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=150,
                temperature=0
            )
            result = response.choices[0].message.content.strip()
            #print(result)

            sentiment_line = next((line for line in result.split('\n') if "label:" in line.lower()), None)
            reason_line = next((line for line in result.split('\n') if "reason:" in line.lower()), None)

            if sentiment_line:
                try:
                    # Extract sentiment value and clean it
                    sentiment_str = sentiment_line.split(":")[1].strip()
                    # Handle negative numbers properly
                    if sentiment_str in ['-1', '0', '1']:
                        sentiment = int(sentiment_str)
                    else:
                        print(f"Invalid sentiment for title: {title}")
                        print(f"Raw sentiment line: {sentiment_line}")
                        print(f"Raw reason line: {reason_line}")
                        raise ValueError(f"Invalid sentiment value: {sentiment_line}")

                    reason = reason_line.split(":", 1)[1].strip()
                    return sentiment, reason

                except ValueError as e:
                    print(f"Warning: Could not parse sentiment: {e}")
                    return -999, "Error parsing sentiment"

            else:
                raise ValueError("Invalid response format")

        except Exception as e:
            print(f"API call failed: {e}. Retrying in 1-2 minutes...")
            time.sleep(random.randint(60, 120))

    return -999, "Failed after 3 retries"

In [ ]:
"""Function to process story titles and save results"""

def process_titles(df, output_csv_file, save_interval=20):
    # Resume from existing file, if available
    try:
        existing_df = pd.read_csv(output_csv_file)
        df.update(existing_df)  # Merge saved progress
        print("Resuming from the last saved point...")
    except FileNotFoundError:
        print("No existing output file found. Starting fresh...")

    pbar = tqdm(total=len(df), desc="Processing story titles")

    for i, row in df.iterrows():
        # Skip rows already processed
        if pd.notna(row.get('senti_prompt0_1shot_gpt')) and pd.notna(row.get('reason_prompt0_1shot_gpt')):
            pbar.update(1)
            continue

        # Get sentiment and reason for the story title
        sentiment, reason = get_sentiment_and_reason_openai(row['title'])
        df.at[i, 'senti_prompt0_1shot_gpt'] = sentiment
        df.at[i, 'reason_prompt0_1shot_gpt'] = reason
        print(f"sentiment: {sentiment}")
        print(f"reason: {reason}")
        pbar.update(1)

        # Save progress at intervals
        if (i + 1) % save_interval == 0:
            df.to_csv(output_csv_file, index=False)
            print(f"\nSaved progress at row {i + 1}")

    pbar.close()

    # Final save
    df.to_csv(output_csv_file, index=False)
    print(f"Final result saved to {output_csv_file}")


### Load the CSV file with the HN GH AI story

In [ ]:
df = pd.read_csv(output_csv_file)

# Add columns if missing
if 'senti_prompt0_1shot_gpt' not in df.columns:
    df['senti_prompt0_1shot_gpt'] = None
if 'reason_prompt0_1shot_gpt' not in df.columns:
    df['reason_prompt0_1shot_gpt'] = None

# Process titles
process_titles(df, output_csv_file)

print(f'Sentiment analysis completed and saved to {output_csv_file}')

# Evaluation

In [8]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

In [4]:
df = pd.read_csv(output_csv_file)

In [5]:
df = df[df["story_consensus"].notna()]

In [9]:
print(f"Accuracy: {accuracy_score(df['story_consensus'], df['senti_prompt0_1shot_gpt'])}")
print(f"micro f1: {f1_score(df['story_consensus'], df['senti_prompt0_1shot_gpt'], average='micro')}")
print(f"macro f1: {f1_score(df['story_consensus'], df['senti_prompt0_1shot_gpt'], average='macro')}")
print(f"weighted f1: {f1_score(df['story_consensus'], df['senti_prompt0_1shot_gpt'], average='weighted')}")

Accuracy: 0.8333333333333334
micro f1: 0.8333333333333334
macro f1: 0.7655172413793103
weighted f1: 0.8747126436781609
